In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

In [2]:
#Load Pre-Processed Train / Test Data

train_df = pd.read_csv('Train-Data.csv')
test_df = pd.read_csv('Test-Data.csv')

print(f'Train Length: {len(train_df)}, Test Length: {len(test_df)}')

Train Length: 1760, Test Length: 440


In [3]:
#Prepare Data For ML input
trainx = np.array(train_df.iloc[:,:-1].values.tolist())
trainy = np.array(train_df['label'].tolist())

testx = np.array(test_df.iloc[:,:-1].values.tolist())
testy = np.array(test_df['label'].tolist())

In [4]:
#Validate Correct Input Shapes
print(f'Trainx: {trainx.shape}, Trainy: {trainy.shape}')
print(f'Testx: {testx.shape}, Testy: {testy.shape}')

Trainx: (1760, 7), Trainy: (1760,)
Testx: (440, 7), Testy: (440,)


In [5]:
#Encode Classification Labels
le = LabelEncoder()

le.fit(trainy)

trainy = le.transform(trainy)
testy = le.transform(testy)

In [6]:
#Build Fully Connected Neural Network Model
FCNN = Sequential()
FCNN.add(Dense(32, input_dim=7, activation='relu'))
FCNN.add(Dense(64, activation='relu'))
FCNN.add(Dense(22, activation='softmax'))
FCNN.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
FCNN.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                256       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dense_2 (Dense)             (None, 22)                1430      
                                                                 
Total params: 3,798
Trainable params: 3,798
Non-trainable params: 0
_________________________________________________________________


In [8]:
FCNN.fit(trainx, trainy, epochs=50, batch_size=32)

Epoch 1/50
55/55 [==============================] - 0s 730us/step - loss: 9.6847 - accuracy: 0.1608
Epoch 2/50
55/55 [==============================] - 0s 652us/step - loss: 1.0857 - accuracy: 0.6534
Epoch 3/50
55/55 [==============================] - 0s 643us/step - loss: 0.5023 - accuracy: 0.8369
Epoch 4/50
55/55 [==============================] - 0s 671us/step - loss: 0.3612 - accuracy: 0.8949
Epoch 5/50
55/55 [==============================] - 0s 680us/step - loss: 0.3084 - accuracy: 0.9062
Epoch 6/50
55/55 [==============================] - 0s 661us/step - loss: 0.2572 - accuracy: 0.9205
Epoch 7/50
55/55 [==============================] - 0s 661us/step - loss: 0.2199 - accuracy: 0.9369
Epoch 8/50
55/55 [==============================] - 0s 661us/step - loss: 0.2068 - accuracy: 0.9426
Epoch 9/50
55/55 [==============================] - 0s 652us/step - loss: 0.2081 - accuracy: 0.9364
Epoch 10/50
55/55 [==============================] - 0s 671us/step - loss: 0.1947 - accuracy: 0.9460

In [9]:
FCNN_Preds = FCNN.predict(testx)
FCNN_Preds = np.array([np.argmax(i) for i in FCNN_Preds])

14/14 [==============================] - 0s 534us/step


In [10]:
def get_performance_metrics(preds):
    metrics = {}
    metrics['acc'] = accuracy_score(testy, preds)
    metrics['prec'] = precision_score(testy, preds, average='macro')
    metrics['rec'] = recall_score(testy, preds, average='macro')
    metrics['f1'] = f1_score(testy, preds, average='macro')
    metrics['cm'] = confusion_matrix(testy, preds)
    return metrics

In [11]:
FCNN_Metrics = get_performance_metrics(FCNN_Preds)
print(FCNN_Metrics['acc'])

0.9431818181818182


In [12]:
#Build CNN
CNN = Sequential()
CNN.add(Conv1D(filters=32, kernel_size=3, input_shape=(7, 1), activation='relu'))
CNN.add(Flatten())
CNN.add(Dense(128, activation='relu'))
CNN.add(Dense(22, activation='softmax'))
CNN.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [13]:
trainx_reshaped = trainx.reshape(trainx.shape[0], 7, 1)
testx_reshaped = testx.reshape(testx.shape[0], 7, 1)
CNN.fit(trainx_reshaped, trainy, epochs=50, batch_size=32)

Epoch 1/50
55/55 [==============================] - 0s 772us/step - loss: 3.2136 - accuracy: 0.4472
Epoch 2/50
55/55 [==============================] - 0s 762us/step - loss: 0.4029 - accuracy: 0.8875
Epoch 3/50
55/55 [==============================] - 0s 744us/step - loss: 0.2327 - accuracy: 0.9364
Epoch 4/50
55/55 [==============================] - 0s 735us/step - loss: 0.1831 - accuracy: 0.9506
Epoch 5/50
55/55 [==============================] - 0s 744us/step - loss: 0.1584 - accuracy: 0.9511
Epoch 6/50
55/55 [==============================] - 0s 744us/step - loss: 0.1496 - accuracy: 0.9511
Epoch 7/50
55/55 [==============================] - 0s 744us/step - loss: 0.1766 - accuracy: 0.9335
Epoch 8/50
55/55 [==============================] - 0s 964us/step - loss: 0.1897 - accuracy: 0.9443
Epoch 9/50
55/55 [==============================] - 0s 753us/step - loss: 0.1423 - accuracy: 0.9563
Epoch 10/50
55/55 [==============================] - 0s 762us/step - loss: 0.1401 - accuracy: 0.9477

In [14]:
CNN_Preds = CNN.predict(testx)
CNN_Preds = np.array([np.argmax(i) for i in CNN_Preds])

14/14 [==============================] - 0s 623us/step


In [15]:
CNN_Metrics = get_performance_metrics(CNN_Preds)
print(CNN_Metrics['acc'])

0.9659090909090909
